In [1]:
from konlpy.tag import Komoran
from konlpy.tag import Kkma
import pandas as pd
import numpy as np
from re import match

In [2]:
data = pd.read_csv("./crawling_result_daum.csv")
# data = pd.read_csv("./crawling_result_naver.csv", 
#                 names=['제목','작가','평점','장르','연령','한줄 스토리'])
story = data[['한줄 스토리']]

In [3]:
komoran = Komoran()
kkma = Kkma()

keywords = []
temp = []
empty_list = [' ']

# kkma와 komoran 두클래스에서 모두 일반명사이고, 길이가 2이상인 단어들
for i in story.index:
    try:
        val = story._get_value(i,"한줄 스토리")
        words = komoran.pos(val)
        for(text, tclass) in words:
            if tclass=='NNG' and not len(text)==1 and not(match('^[0-9]',text)):
                temp.append(text)
        words = kkma.pos(val)
        for(text, tclass) in words:
            if tclass=='NNG' and not len(text)==1 and not(match('^[0-9]',text)):
                temp.append(text)
        keywords.append(temp.copy())
        temp.clear()
        if (i%10)==0:
            print(i,' ', end='')
    except:
        keywords.append(empty_list.copy())

#중복제거
newkeywords = []
for i in range(len(keywords)):
    newkeywords.append(list(set(keywords[i])))

0  10  20  30  40  50  60  70  80  90  100  110  120  130  140  150  160  170  180  190  200  210  220  230  240  250  260  270  280  290  300  310  320  330  340  350  360  370  380  390  400  410  420  430  440  450  460  470  480  490  500  510  520  530  540  550  560  570  580  590  600  610  620  630  640  650  660  670  680  690  700  710  720  730  740  750  760  770  780  790  800  810  820  830  840  850  860  870  880  890  900  910  920  930  940  950  960  970  980  990  1000  1010  1020  1030  1040  1050  1060  1070  1080  1090  1100  1110  1120  1130  1140  1150  1160  1170  1180  1190  1200  

In [7]:
# 삭제할 특정 단어 리스트
delete_words = ['이의', '작품', '만화','남자','여자', '자신', '스스로', 
                '의의' ,'사이','남성','여성','만이','작가', '인간', '사람', 
                '시작', '신작', '이번']

for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in delete_words:
            newkeywords[i].remove(v)

for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in delete_words:
            newkeywords[i].remove(v)

In [8]:
for i in range(len(newkeywords)):
    print(i,': ', end='')
    print(newkeywords[i])

0 : ['한철', '동물', '유명', '전부', '소리', '수의사', '성공']
1 : ['좌충우돌', '공주', '모험', '로맨스', '개척', '운명']
2 : ['선녀', '판타지', '로맨스', '달님', '싸가지', '지상', '최대']
3 : ['생활', '대학생', '뱀파이어', '시영', '동거']
4 : ['현세', '일상', '이야기', '리머', '소통']
5 : ['협객', '무영', '속물', '사항', '희망', '사부']
6 : ['첫사랑']
7 : ['친구', '햇살', '만들기', '프로젝트', '하나', '미소년']
8 : ['일상', '수학', '대학생', '쓰레기']
9 : ['호구', '탐욕', '몸부림', '주식', '사기극']
10 : ['민성', '연애', '사건', '예기', '비밀']
11 : ['교수', '교수님', '수업', '수강생', '보컬']
12 : ['공포증', '이야기', '가족', '조류', '돌연변이']
13 : ['막장', '학교', '문제아', '사상', '정문', '최악', '쓰레기', '여상']
14 : ['시스트', '소녀', '이야기']
15 : ['활극', '라마', '세상', '오리', '액션']
16 : ['생활', '지름', '소한', '오라', '세계']
17 : ['생존', '싸움', '트레이스', '트러블']
18 : ['최선', '세상', '조경']
19 : ['싸움판', '저마다', '소년', '사정']
20 : ['고군', '분투', '이야기', '톱스타', '고군분투', '사랑', '신랑']
21 : ['만남', '의미', '과의', '변화', '영진', '전학생', '비밀', '유은']
22 : ['유명', '인터넷', '살인', '편집', '방송인', '진실', '사건']
23 : ['지망생', '방송', '선수', '지망', '유도', '졸업']
24 : ['아빠', '소유', '수술', '남자애', '심장', '이식']
25 : ['순간', '로맨스',

506 : ['귀신', '환상', '담임', '무의식', '커플', '생과', '선생님', '형상화', '여고']
507 : ['고통', '재벌', '인생', '상속']
508 : ['소년원', '방황', '대신', '친구']
509 : ['아빠', '정도', '삼각관계', '공포증', '짝사랑']
510 : ['주인', '추방', '악마', '사랑', '제임스']
511 : ['라이프', '감수성', '깨알', '디테일', '두부']
512 : ['불안', '나비', '채팅', '랜덤', '호감', '비밀']
513 : ['막걸리', '대결', '최고', '신명']
514 : ['다음']
515 : ['환상적', '환상', '남녀', '운명적', '이야기', '아래', '사랑', '운명']
516 : ['사랑', '심장', '동시', '세상']
517 : ['속물', '미남']
518 : ['오토바이', '사랑', '기대어', '생각']
519 : ['불행', '소년', '드로', '부유', '소녀', '이드']
520 : ['작업', '동경']
521 : ['중국', '여행', '요리', '광저우', '탐방', '동아리', '세계', '미식']
522 : ['사랑', '거절']
523 : ['임무', '생존', '수행', '가족']
524 : ['남매', '중심', '엄친', '교실', '전학']
525 : ['소녀', '앵초', '소년', '이야기']
526 : ['초짜', '사칭', '이야기', '무림', '고수']
527 : ['침입', '애정', '스토킹', '원한', '무단', '사실']
528 : ['동물', '단과', '사건', '아람']
529 : ['대학교', '입학', '성적']
530 : ['악연', '굴레', '지금', '시간', '인연']
531 : ['성녀', '능력', '건달', '육체']
532 : ['투쟁', '공생', '소년', '이야기', '무신', '왕따', '전설적', '사랑']
533 : ['스토리', '하늘', '지

845 : ['실전', '노트', '연애', '호원', '제일']
846 : ['기상천외', '일우', '소녀', '연하', '동거']
847 : ['마음', '이상', '담임']
848 : ['과학', '모험', '드라마', '공상', '데뷔작']
849 : ['하루하루', '청춘', '이야기']
850 : ['소년', '지독', '사후', '사랑', '소녀', '세계']
851 : ['스타일', '지금']
852 : ['죽음', '승자', '대립', '소녀', '인격', '게임']
853 : ['공학', '미래', '바이오', '거대', '음모', '발달', '서울']
854 : ['해경', '저녁', '대화']
855 : ['사랑', '제대', '지금']
856 : ['이상', '판타지', '상상', '드라마', '무엇', '개념', '액션']
857 : ['심장', '일상', '기억', '순간']
858 : ['아이', '괴물', '아이돌']
859 : ['영업', '귀신', '식당', '초대']
860 : ['정체', '학생', '미소녀', '딸기']
861 : ['일자리', '통일', '주세', '서울']
862 : ['오타쿠', '취미', '대한민국', '외국산']
863 : ['기억']
864 : ['모두', '개발', '궁극', '기구', '놀이']
865 : ['대왕', '향연', '파워', '농촌']
866 : ['계속', '아웃', '전체', '도시', '화이트']
867 : ['가상현실', '아웃', '로그', '게임', '위험', '유저', '시티', '언더']
868 : ['젊은이', '권장', '나라']
869 : ['산악', '구조', '본적', '이드', '액션']
870 : ['조금', '이야기', '식구', '어가', '중반', '동재']
871 : ['동물', '반려', '메트로놈', '조금', '이야기']
872 : ['화성']
873 : ['맞춤', '양복점']
874 : ['개인', '서울대', '나태', '한의']


In [16]:
data['스토리 키워드'] = newkeywords

In [17]:
data.to_csv("./storykeywords_result_daum.csv", header=True, index = False)
# data.to_csv("./storykeywords_result_naver.csv", header=True, index = False)